# End to End CLV Model Notebook

# Getting Started with Snowflake Feature Store
We will use the Use-Case to show how Snowflake Feature Store (and Model Registry) can be used to maintain & store features, retrieve them for training and perform micro-batch inference.

In the development (TRAINING) enviroment we will 
- create FeatureViews in the Feature Store that maintain the required customer-behaviour features.
- use these Features to train a model, and save the model in the Snowflake model-registry.
- plot the clusters for the trained model to visually verify. 

In the production (SERVING) environment we will
- re-create the FeatureViews on production data
- generate an Inference FeatureView that uses the saved model to perform incremental inference

# Feature Engineering & Model Training

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Notebook Packages

In [8]:
# Python packages
import os
import json
import timeit

# SNOWFLAKE
# Snowpark
from snowflake.snowpark import Session, DataFrame, Window, WindowSpec

import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

# Snowflake Feature Store
from snowflake.ml.feature_store import (
    FeatureView,
    Entity)

# COMMON FUNCTIONS
from helper.useful_fns import dataset_check_and_update, formatSQL, create_ModelRegistry, create_FeatureStore, create_SF_Session, get_spine_df

### Setup Snowflake connection and database parameters

In [9]:
role_name, database_name, schema_name, session, warehouse_name = create_SF_Session(connection_file = '../../connection.json')

You might have more than one threads sharing the Session object trying to update sql_simplifier_enabled. Updating this while other tasks are running can potentially cause unexpected behavior. Please update the session configuration before starting the threads.



Connection Established with the following parameters:
User                        : JARCHEN
Role                        : "RETAIL_REGRESSION_DEMO_ROLE"
Database                    : "RETAIL_REGRESSION_DEMO"
Schema                      : "DS"
Warehouse                   : "RETAIL_REGRESSION_DEMO_WH"
Snowflake version           : 9.38.4
Snowpark for Python version : 1.38.0 



## MODEL DEVELOPMENT
* Create Snowflake Model-Registry
* Create Snowflake Feature-Store
* Establish and Create CUSTOMER Entity in the development Snowflake FeatureStore
* Create Source Data references and perform basic data-cleansing
* Create & Run Preprocessing Function to create features
* Create FeatureView_Preprocess from Preprocess Dataframe SQL
* Create training data from FeatureView_Preprocess (asof join)
* Create & Fit Snowpark-ml pipeline 
* Save model in Model Registry
* 'Verify' and approve model
* Create new FeatureView_Model_Inference with Transforms UDF + KMeans model

In [14]:
# Create/Reference Snowflake Model Registry - Common across Environments
mr = create_ModelRegistry(session, database_name, '_MODELLING')

# Create/Reference Snowflake Feature Store - Common across Environments
fs = create_FeatureStore(session, database_name, '_FEATURE_STORE', warehouse_name)


Model Registry (_MODELLING) already exists
Feature Store (_FEATURE_STORE) created


In [15]:
cust_tbl = '.'.join([database_name, schema_name,'CUSTOMERS'])
cust_sdf = session.table(cust_tbl)
print(cust_tbl, cust_sdf.count())
cust_sdf.limit(10).show()

RETAIL_REGRESSION_DEMO.DS.CUSTOMERS 10000
---------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"AGE"  |"ANNUAL_INCOME"  |"LOYALTY_TIER"  |"TENURE_MONTHS"  |"SIGNUP_DATE"  |"CREATED_AT"                |
---------------------------------------------------------------------------------------------------------------------------
|1              |32     |43903.00         |2               |18               |2024-06-09     |2025-12-09 01:11:59.639000  |
|2              |36     |20198.00         |1               |58               |2021-02-09     |2025-12-09 01:11:59.639000  |
|3              |41     |56364.00         |4               |34               |2023-02-09     |2025-12-09 01:11:59.639000  |
|4              |50     |51223.00         |2               |18               |2024-06-09     |2025-12-09 01:11:59.639000  |
|5              |32     |25744.00         |2               |3                |2025-09-09  

### CUSTOMER Entity
Establish and Create CUSTOMER Entity in Snowflake FeatureStore for this Use-Case

In [16]:
if "CUSTOMER" not in json.loads(fs.list_entities().select(F.to_json(F.array_agg("NAME", True))).collect()[0][0]):
    customer_entity = Entity(name="CUSTOMER", join_keys=["CUSTOMER_ID"],desc="Primary Key for CUSTOMER ORDER")
    fs.register_entity(customer_entity)
else:
    customer_entity = fs.get_entity("CUSTOMER")

fs.list_entities().show()

---------------------------------------------------------------------------------------------
|"NAME"    |"JOIN_KEYS"      |"DESC"                          |"OWNER"                      |
---------------------------------------------------------------------------------------------
|CUSTOMER  |["CUSTOMER_ID"]  |Primary Key for CUSTOMER ORDER  |RETAIL_REGRESSION_DEMO_ROLE  |
---------------------------------------------------------------------------------------------



 ### Create & Load Source Data

Our Feature engineering pipelines are defined using Snowpark dataframes (or SQL expressions).  In the `QS_feature_engineering_fns.py` file we have created two feature engineering functions to create our pipeline :
* __uc01_load_data__(order_data: DataFrame, lineitem_data: DataFrame, order_returns_data: DataFrame) -> DataFrame   
* __uc01_pre_process__(data: DataFrame) -> DataFrame

`uc01_load_data`, takes the source tables, as dataframe objects, and joins them together, performing some data-cleansing by replacing NA's with default values. It returns a dataframe as it's output.

`uc01_pre_process`, takes the dataframe output from `uc01_load_data`  and performs aggregation on it to derive some features that will be used in our segmentation model.  It returns a dataframe as output, which we will use to provide the feature-pipeline definition within our FeatureView.

In this way we can build up a complex pipeline step-by-step and use it to derive a FeatureView, that will be maintained as a pipeline in Snowflake.

We will import the functions, and create dataframes from them using the dataframes we created earlier pointing to the tables in our TRAINING (Development) schema.  We will use the last dataframe we create at the end of the pipeline as our input to the FeatureView.


In [98]:
# Feature Engineering Functions
from feature_engineering_fns import uc01_load_data, uc01_pre_process

In [99]:
# Tables
cust_tbl                    = '.'.join([database_name, schema_name,'CUSTOMERS'])
behavior_tbl                = '.'.join([database_name, schema_name,'PURCHASE_BEHAVIOR'])

# Snowpark Dataframe
cust_sdf              = session.table(cust_tbl)
behavior_tbl          = session.table(behavior_tbl)

# Row Counts
print(f'''\nTABLE ROW_COUNTS IN {schema_name}''')
print(cust_tbl, cust_sdf.count())
print(behavior_tbl, behavior_tbl.count())


TABLE ROW_COUNTS IN DS
RETAIL_REGRESSION_DEMO.DS.CUSTOMERS 10000
<snowflake.snowpark.table.Table object at 0x303283ac0> 10000


In [100]:
raw_data = uc01_load_data(cust_sdf, behavior_tbl)

In [101]:
# Format and print the SQL for the Snowpark Dataframe
rd_sql = formatSQL(raw_data.queries['queries'][0], True)
print(os.linesep.join(rd_sql.split(os.linesep)[:1000]))

WITH SNOWPARK_LEFT AS (
  SELECT
    "CUSTOMER_ID" AS "l_0026_CUSTOMER_ID",
    "AGE" AS "AGE",
    "ANNUAL_INCOME" AS "ANNUAL_INCOME",
    "LOYALTY_TIER" AS "LOYALTY_TIER",
    "TENURE_MONTHS" AS "TENURE_MONTHS",
    "SIGNUP_DATE" AS "SIGNUP_DATE",
    "CREATED_AT" AS "l_0026_CREATED_AT"
  FROM RETAIL_REGRESSION_DEMO.DS.CUSTOMERS
), SNOWPARK_RIGHT AS (
  SELECT
    "CUSTOMER_ID" AS "r_0027_CUSTOMER_ID",
    "AVG_ORDER_VALUE" AS "AVG_ORDER_VALUE",
    "PURCHASE_FREQUENCY" AS "PURCHASE_FREQUENCY",
    "RETURN_RATE" AS "RETURN_RATE",
    "LIFETIME_VALUE" AS "LIFETIME_VALUE",
    "LAST_PURCHASE_DATE" AS "LAST_PURCHASE_DATE",
    "TOTAL_ORDERS" AS "TOTAL_ORDERS",
    "CREATED_AT" AS "r_0027_CREATED_AT"
  FROM RETAIL_REGRESSION_DEMO.DS.PURCHASE_BEHAVIOR
), cte AS (
  SELECT
    *
  FROM (
    SNOWPARK_LEFT AS SNOWPARK_LEFT
      LEFT OUTER JOIN SNOWPARK_RIGHT AS SNOWPARK_RIGHT
        ON (
          "l_0026_CUSTOMER_ID" = "r_0027_CUSTOMER_ID"
        )
  )
), cte_2 AS (
  SELECT
    *
    R

In [102]:
raw_data.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"AGE"  |"ANNUAL_INCOME"  |"LOYALTY_TIER"  |"TENURE_MONTHS"  |"SIGNUP_DATE"  |"CREATED_AT"                |"AVG_ORDER_VALUE"  |"PURCHASE_FREQUENCY"  |"RETURN_RATE"  |"LIFETIME_VALUE"  |"LAST_PURCHASE_DATE"  |"TOTAL_ORDERS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1              |32     |43903.00         |2               |18               |2024-06-09     |2025-12-09 01:11:59.639000  |212.78             |4.09                  |17.00          |16559.17          |2025-11-22            |74              |
|2              |36     |20198.0

### Create & Run Preprocessing Function 

In [103]:
preprocessed_data = uc01_pre_process(raw_data)

In [104]:
preprocessed_data.show()

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"CUSTOMER_ID"  |"AGE"  |"LOYALTY_TIER"  |"TENURE_MONTHS"  |"SIGNUP_DATE"  |"CREATED_AT"                |"AVG_ORDER_VALUE"  |"PURCHASE_FREQUENCY"  |"RETURN_RATE"  |"LIFETIME_VALUE"  |"LAST_PURCHASE_DATE"  |"TOTAL_ORDERS"  |"ANNUAL_INCOME"  |"AVERAGE_ORDER_PER_MONTH"  |"DAYS_SINCE_LAST_PURCHASE"  |"DAYS_SINCE_SIGNUP"  |"EXPECTED_DAYS_BETWEEN_PURCHASES"  |"DAYS_SINCE_EXPECTED_LAST_PURCHASE_DATE"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [105]:
# Format and print the SQL for the Snowpark Dataframe
ppd_sql = formatSQL(preprocessed_data.queries['queries'][0], True)
print(os.linesep.join(ppd_sql.split(os.linesep)[:1000]))

WITH SNOWPARK_LEFT AS (
  SELECT
    "CUSTOMER_ID" AS "l_0026_CUSTOMER_ID",
    "AGE" AS "AGE",
    "ANNUAL_INCOME" AS "ANNUAL_INCOME",
    "LOYALTY_TIER" AS "LOYALTY_TIER",
    "TENURE_MONTHS" AS "TENURE_MONTHS",
    "SIGNUP_DATE" AS "SIGNUP_DATE",
    "CREATED_AT" AS "l_0026_CREATED_AT"
  FROM RETAIL_REGRESSION_DEMO.DS.CUSTOMERS
), SNOWPARK_RIGHT AS (
  SELECT
    "CUSTOMER_ID" AS "r_0027_CUSTOMER_ID",
    "AVG_ORDER_VALUE" AS "AVG_ORDER_VALUE",
    "PURCHASE_FREQUENCY" AS "PURCHASE_FREQUENCY",
    "RETURN_RATE" AS "RETURN_RATE",
    "LIFETIME_VALUE" AS "LIFETIME_VALUE",
    "LAST_PURCHASE_DATE" AS "LAST_PURCHASE_DATE",
    "TOTAL_ORDERS" AS "TOTAL_ORDERS",
    "CREATED_AT" AS "r_0027_CREATED_AT"
  FROM RETAIL_REGRESSION_DEMO.DS.PURCHASE_BEHAVIOR
), cte AS (
  SELECT
    *
  FROM (
    SNOWPARK_LEFT AS SNOWPARK_LEFT
      LEFT OUTER JOIN SNOWPARK_RIGHT AS SNOWPARK_RIGHT
        ON (
          "l_0026_CUSTOMER_ID" = "r_0027_CUSTOMER_ID"
        )
  )
), cte_2 AS (
  SELECT
    *
    R

### Create Preprocessing FeatureView from Preprocess Dataframe (SQL)

In [ ]:
# Define descriptions for the FeatureView's Features.  These will be added as comments to the database object
preprocess_features_desc = {  
   "AVERAGE_ORDER_PER_MONTH":"Average number of orders per month",
   "DAYS_SINCE_LAST_PURCHASE":"Days since last purchase",
   "DAYS_SINCE_SIGNUP":"Days since signup",
   "EXPECTED_DAYS_BETWEEN_PURCHASES":"Expected days between purchases",
   "DAYS_SINCE_EXPECTED_LAST_PURCHASE_DATE":"Days since expected last purchase date from LAST_PURCHASE_DATE"
}

ppd_fv_name    = "FV_PREPROCESS"
ppd_fv_version = "V_1"

try:
   # If FeatureView already exists just return the reference to it
   fv_uc01_preprocess = fs.get_feature_view(name=ppd_fv_name,version=ppd_fv_version)
except:
   # Create the FeatureView instance
   fv_uc01_preprocess_instance = FeatureView(
      name=ppd_fv_name, 
      entities=[customer_entity], 
      feature_df=preprocessed_data,      # <- We can use the snowpark dataframe as-is from our Python
      # feature_df=preprocessed_data.queries['queries'][0],    # <- Or we can use SQL, in this case linted from the dataframe generated SQL to make more human readable
      timestamp_col="LATEST_ORDER_DATE",
      refresh_freq="60 minute",            # <- specifying optional refresh_freq creates FeatureView as Dynamic Table, else created as View.
      desc="Features to support Use Case 01").attach_feature_desc(preprocess_features_desc)

   # Register the FeatureView instance.  Creates  object in Snowflake
   fv_uc01_preprocess = fs.register_feature_view(
      feature_view=fv_uc01_preprocess_instance, 
      version=ppd_fv_version, 
      block=True,     # whether function call blocks until initial data is available
      overwrite=False # whether to replace existing feature view with same name/version
   )
   print(f"Feature View : {ppd_fv_name}_{ppd_fv_version} created")   
else:
   print(f"Feature View : {ppd_fv_name}_{ppd_fv_version} already created")
finally:
   fs.list_feature_views().show(20)
spine = fv_uc01_preprocess

Feature View : FV_UC01_PREPROCESS_V_1 already created
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"NAME"              |"VERSION"  |"DATABASE_NAME"               |"SCHEMA_NAME"            |"CREATED_ON"                |"OWNER"     |"DESC"                           |"ENTITIES"  |"REFRESH_FREQ"  |"REFRESH_MODE"  |"SCHEDULING_STATE"  |"WAREHOUSE"                  |"CLUSTER_BY"                            |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|FV_UC01_PREPROCESS  |V_1

In [17]:
# You can also use the following to retrieve a Feature View instance for use within Python
FV_UC01_PREPROCESS_V_1 = fs.get_feature_view(ppd_fv_name, 'V_1')

In [18]:
# We can look at the FeatureView's contents with
FV_UC01_PREPROCESS_V_1.feature_df.sort(F.col("O_CUSTOMER_SK"), ascending=False).show(10)

---------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"LATEST_ORDER_DATE"  |"FREQUENCY"  |"RETURN_RATIO"  |"RETURN_ROW_PRICE"  |
---------------------------------------------------------------------------------------------
|70709            |2024-04-27           |2.000        |1.000           |14.000              |
|70708            |2024-08-17           |2.000        |1.000           |85.000              |
|70707            |2024-05-30           |4.333        |1.000           |23.000              |
|70706            |2024-06-16           |3.000        |1.000           |41.000              |
|70705            |2024-03-14           |1.667        |1.000           |17.000              |
|70704            |2024-02-16           |3.333        |1.000           |18.000              |
|70703            |2024-06-29           |3.333        |1.000           |36.000              |
|70702            |2024-07-06           |2.000        |1.000

### Create training data Dataset from FeatureView_Preprocess

In [19]:
# Create Spine
spine_sdf = get_spine_df(spine)
spine_sdf.sort('O_CUSTOMER_SK').show(5)

-------------------------------------------
|"O_CUSTOMER_SK"  |"ASOF_DATE"  |"COL_1"  |
-------------------------------------------
|0                |2024-08-02   |values1  |
|1                |2024-05-02   |values1  |
|2                |2024-05-24   |values1  |
|3                |2024-07-18   |values1  |
|4                |2024-02-23   |values1  |
-------------------------------------------



In [ ]:
def generate_training_df(spine_sdf, feature_view):
    dataset_name = 'UC01_TRAINING'
    dataset_version = dataset_check_and_update(session, dataset_name)
    # Generate_Dataset
    training_dataset = fs.generate_dataset( 
        name = dataset_name,
        version = dataset_version,
        spine_df = spine_sdf, 
        features = [feature_view], 
        spine_timestamp_col = 'ASOF_DATE'
        )                                     
    # Create a snowpark dataframe reference from the Dataset
    training_dataset_sdf = training_dataset.read.to_snowpark_dataframe()
    
    return training_dataset_sdf

In [ ]:
# Generate_Dataset
training_dataset_sdf_v1 = generate_training_df(spine_sdf, fv_uc01_preprocess)
# Display some sample data
training_dataset_sdf_v1.sort('O_CUSTOMER_SK').show(5)

------------------------------------------------------------------------------------------------------
|"O_CUSTOMER_SK"  |"ASOF_DATE"  |"COL_1"  |"FREQUENCY"         |"RETURN_RATIO"  |"RETURN_ROW_PRICE"  |
------------------------------------------------------------------------------------------------------
|0                |2024-08-02   |values1  |1.6670000553131104  |1.0             |69.0                |
|1                |2024-05-02   |values1  |2.3329999446868896  |1.0             |39.0                |
|2                |2024-05-24   |values1  |1.6670000553131104  |1.0             |27.0                |
|3                |2024-07-18   |values1  |3.0                 |1.0             |65.0                |
|4                |2024-02-23   |values1  |2.3329999446868896  |1.0             |12.0                |
------------------------------------------------------------------------------------------------------



In [23]:
training_dataset_sdf_v1.to_pandas().head()

,O_CUSTOMER_SK,ASOF_DATE,COL_1,FREQUENCY,RETURN_RATIO,RETURN_ROW_PRICE
0,57584,2024-07-04,values1,1.500,1.0,17.0
1,9441,2024-05-29,values1,2.000,1.0,154.0
2,46022,2024-05-30,values1,4.000,1.0,29.0
3,2448,2024-04-15,values1,2.333,1.0,65.0
4,51062,2024-07-31,values1,2.000,1.0,60.0


## CLEAN UP

In [ ]:
# session.close()

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
formatted_time = datetime.now(ZoneInfo("Australia/Melbourne")).strftime("%A, %B %d, %Y %I:%M:%S %p %Z")

print(f"The last run time in Melbourne is: {formatted_time}")

The last run time in Melbourne is: Friday, November 21, 2025 12:03:58 AM AEDT
